# Simple, Phase 1, approximate flight profile simulation

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import control

# Purdue AA451 library
from scripts.jsbsim_utils import Logger, trim, simulate, linearize, rootlocus, clean_tf

In [108]:
aircraft_model = 'WaveRipper'

# Ground Trim

In [112]:
op_ground, props = trim(
    aircraft=aircraft_model,
    ic={
        'ic/vt-fps': 0,
        'ic/h-agl-ft': 5,
        'ic/psi-true-deg': 280,
        'fcs/left-brake-cmd-norm': 1,
        'fcs/right-brake-cmd-norm': 1,
        'fcs/center-brake-cmd-norm': 1,
    },
    design_vector=['ic/theta-rad', 'ic/h-agl-ft'],
    x0=[0, 0],
    verbose=True,
    method='Nelder-Mead', # works better with ground interaction
    ftol=1e-1,
)
op_ground

 final_simplex: (array([[-0.0036453 , -0.00151608],
       [-0.0036714 , -0.00144496],
       [-0.00367275, -0.00142257]]), array([0.09572609, 0.09572946, 0.09573725]))
           fun: 0.095726085184104
       message: 'Optimization terminated successfully.'
          nfev: 55
           nit: 29
        status: 0
       success: True
             x: array([-0.0036453 , -0.00151608])



  The aerodynamic moment axis system has been set by default to the bodyXYZ system.


{'ic/vt-fps': 0,
 'ic/h-agl-ft': -0.0015160835740243772,
 'ic/psi-true-deg': 280,
 'fcs/left-brake-cmd-norm': 1,
 'fcs/right-brake-cmd-norm': 1,
 'fcs/center-brake-cmd-norm': 1,
 'ic/theta-rad': -0.003645302904478732}

In [113]:
op_ground

{'ic/vt-fps': 0,
 'ic/h-agl-ft': -0.0015160835740243772,
 'ic/psi-true-deg': 280,
 'fcs/left-brake-cmd-norm': 1,
 'fcs/right-brake-cmd-norm': 1,
 'fcs/center-brake-cmd-norm': 1,
 'ic/theta-rad': -0.003645302904478732}

In [114]:
log_ground = simulate(
    aircraft=aircraft_model,
    op_0=op_ground,
    tf=5,
    realtime=False, verbose=True)


  The aerodynamic moment axis system has been set by default to the bodyXYZ system.


## Cruise Trim

In [116]:
op_cruise, props = trim(
    aircraft=aircraft_model,
    # For a list of initial conditions available:
    # https://jsbsim-team.github.io/jsbsim/classJSBSim_1_1FGInitialCondition.html
    ic={
        'ic/gamma-deg': 0,                          # Flightpath angle initial condition in degrees
        'ic/vt-fps': 600,                           # True airspeed initial condition in feet/second
        'ic/h-sl-ft': 10000,                        # Height above sea level initial condition in feet
        'gear/gear-cmd-norm': 0,
        'propulsion/engine/pitch-angle-rad': 0,
    },
    design_vector=[
        'fcs/throttle-cmd-norm',
        'fcs/elevator-cmd-norm',
        'fcs/rudder-cmd-norm',
        'fcs/aileron-cmd-norm',
        'ic/alpha-rad',
        'ic/beta-rad',
    ],
    method='SLSQP',
    eq_constraints= [
        lambda fdm: fdm['accelerations/udot-ft_sec2'],
        lambda fdm: fdm['accelerations/vdot-ft_sec2'],
        lambda fdm: fdm['accelerations/wdot-ft_sec2'],
        lambda fdm: fdm['accelerations/pdot-rad_sec2'],
        lambda fdm: fdm['accelerations/qdot-rad_sec2'],
        lambda fdm: fdm['accelerations/rdot-rad_sec2'],
    ],
    cost=lambda fdm: fdm['fcs/throttle-cmd-norm'],
    x0=[0.5, 0, 0, 0, 0, 0],
    verbose=False,
    bounds=[[0, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]],
    tol=1e-3
)
op_cruise


  The aerodynamic moment axis system has been set by default to the bodyXYZ system.


RuntimeError: trim failed:
     fun: 0.5
     jac: array([1., 0., 0., 0., 0., 0.])
 message: 'Singular matrix C in LSQ subproblem'
    nfev: 7
     nit: 1
    njev: 1
  status: 6
 success: False
       x: array([0.5, 0. , 0. , 0. , 0. , 0. ])
{'accelerations/a-pilot-x-ft_sec2': 14.88468803876103, 'accelerations/a-pilot-y-ft_sec2': -1.062270911025521, 'accelerations/a-pilot-z-ft_sec2': -5.3178335117001145, 'accelerations/n-pilot-x-norm': 0.46263024725205476, 'accelerations/n-pilot-y-norm': -0.033016389254289576, 'accelerations/n-pilot-z-norm': -0.16528331839784177, 'accelerations/Nx': 0.462630247625398, 'accelerations/Ny': -0.024112211638101243, 'accelerations/Nz': 0.12021376293816591, 'accelerations/pdot-rad_sec2': 2.584833763975e-09, 'accelerations/qdot-rad_sec2': 0.6306997621480777, 'accelerations/rdot-rad_sec2': -0.12460435154496835, 'accelerations/udot-ft_sec2': 14.884675438581564, 'accelerations/vdot-ft_sec2': -0.7191321948201836, 'accelerations/wdot-ft_sec2': 28.326179640783543, 

In [ ]:
log_cruise = simulate(
    aircraft='F-35B-2',
    op_0=op_cruise,
    tf=10,
    realtime=False)

In [ ]:
op_cruise_auto = dict(op_cruise)
#op_cruise_auto['ic/theta-deg'] = 0
#op_cruise_auto['ic/phi-deg'] = 0
op_cruise_auto['ap/roll-enable'] = 1
op_cruise_auto['ap/pitch-enable'] = 1
op_cruise_auto['ap/yaw-enable'] = 1
op_cruise_auto['ap/h-enable'] = 1
op_cruise_auto['ap/heading-cmd-deg'] = 260
op_cruise_auto['ap/h-sl-cmd-ft'] = 10000

log_cruise_auto = simulate(
    aircraft='F-35B-2',
    op_0=op_cruise_auto,
    tf=40,
    realtime=False)